In [1]:
import seaborn as sns
import torch
import torch.nn as nn


iris = sns.load_dataset("iris")

train=iris.sample(frac=0.8,random_state=42)
test=iris.drop(train.index)

# SEPARANDO AS COLUNAS
x_train = train.loc[ : , ["sepal_length", "sepal_width", "petal_length", "petal_width"]].values
x_test  = test.loc[ : , ["sepal_length", "sepal_width", "petal_length", "petal_width"]].values

# TRANSFORMANDO EM NÚMERO
train["species"] = train.species.map({'setosa':0,'versicolor':1,'virginica':2})
test["species"]  = test.species.map({'setosa':0,'versicolor':1,'virginica':2})

# TARGET TREINO E TESTE
y_train = train.species.values
y_test  = test.species.values

# X
x_train = torch.tensor(x_train, dtype = torch.float)
x_test  = torch.tensor(x_test, dtype = torch.float)
# Y
y_train = torch.tensor(y_train, dtype = torch.long)
y_test  = torch.tensor(y_test, dtype = torch.long)

# NORMALIZANDO
x_train = (x_train - x_train.min())/(x_train.max() - x_train.min())
x_test  = (x_test  - x_test.min())/(x_test.max() - x_test.min())


class NeuralNetwork(nn.Module):
    def __init__(self,input_dim,output_dim):
        super(NeuralNetwork,self).__init__()
        self.input_layer    = nn.Linear(input_dim,128)
        self.hidden_layer1  = nn.Linear(128,64)
        self.output_layer   = nn.Linear(64,output_dim)
        self.relu = nn.ReLU()


    def forward(self,x):
        out =  self.relu(self.input_layer(x))
        out =  self.relu(self.hidden_layer1(out))
        out =  self.output_layer(out)
        return out


input_dim  = 4 ## Quatro características
output_dim = 3 ## Três Espécies
model = NeuralNetwork(input_dim,output_dim)

loss_fn = nn.CrossEntropyLoss()

learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

epochs = 10000

batch_size = 50
batch_no = len(x_train) // batch_size

for epoch in range(epochs):
    for i in range(batch_no):

        start = i * batch_size
        end = start + batch_size

        x_var = x_train[start:end]
        y_var = y_train[start:end]

        optimizer.zero_grad()

        output_train = model(x_var)

        loss = loss_fn(output_train, y_var)

        if (epoch % (epochs / 10) == 0 and i == 0) :
            print(epoch, loss.item())

        loss.backward()

        optimizer.step()

y_pred = torch.argmax(model(x_test), dim = 1)

print(y_pred, y_test)

print((y_pred == y_test).sum()/len(y_test))

0 1.0939018726348877
1000 0.036327723413705826
2000 0.03579600527882576
3000 0.029503436759114265
4000 0.025849001482129097
5000 0.00021669748821295798
6000 3.3463918953202665e-05
7000 7.983807336131576e-06
8000 2.3054415123624494e-06
9000 6.914077630426618e-07
tensor([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 2, 2,
        2, 2, 2, 2, 2, 2]) tensor([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2])
tensor(0.9667)


In [6]:
import seaborn as sns
import torch
import torch.nn as nn


diamonds = sns.load_dataset("diamonds")


# NÃO HÁ NULOS, POR ISSO NÃO PRECISA TRATAR
print('Tem nulos? ' + str(diamonds.isna().any().any()))


# MAPS
map_cut = {'Fair':0,'Good':1,'Very Good':2,'Premium':3,'Ideal':4}
map_color = {'J':0,'I':1,'H':2,'G':3,'F':4,'E':5,'D':6}
map_clarity = {'I1':0,'SI2':1,'SI1':2,'VS2':3,'VS1':4,'VVS2':5,'VVS1':6,'IF':7}


train=diamonds.sample(frac=0.8,random_state=42)
test=diamonds.drop(train.index)


# TRANSFORMANDO EM NÚMERO
train["cut"] = train.cut.map(map_cut)
test["cut"]  = test.cut.map(map_cut)

train["color"] = train.color.map(map_color)
test["color"]  = test.color.map(map_color)

train["clarity"] = train.clarity.map(map_clarity)
test["clarity"]  = test.clarity.map(map_clarity)


# SEPARANDO AS COLUNAS
x_train = train.loc[ : ,  ["carat","cut","color","clarity","depth","table","x","y","z"]].values
x_test  = test.loc[ : , ["carat","cut","color","clarity","depth","table","x","y","z"]].values


# COLUNA QUE EU QUERO PREVER -> price -> dado um diamante, qual o preço dele?
# TARGET TREINO E TESTE
y_train = train.price.values
y_test  = test.price.values


# X
x_train = torch.tensor(x_train, dtype = torch.float)
x_test  = torch.tensor(x_test, dtype = torch.float)
# Y
y_train = torch.tensor(y_train, dtype = torch.float)
y_test  = torch.tensor(y_test, dtype = torch.float)


# NORMALIZANDO
x_train = (x_train - x_train.min())/(x_train.max() - x_train.min())
x_test  = (x_test  - x_train.min())/(x_train.max() - x_train.min())


class NeuralNetwork(nn.Module):
    def __init__(self,input_dim,output_dim):
        super(NeuralNetwork,self).__init__()
        self.input_layer    = nn.Linear(input_dim,128)
        self.hidden_layer1  = nn.Linear(128,64)
        self.output_layer   = nn.Linear(64,output_dim)
        self.relu = nn.ReLU()


    def forward(self,x):
        out =  self.relu(self.input_layer(x))
        out =  self.relu(self.hidden_layer1(out))
        return  self.output_layer(out)


input_dim  = 9 ## Quatro características
output_dim = 1 ## PREÇO FINAL
model = NeuralNetwork(input_dim,output_dim)

# PROBLEMA NÃO É DE CLASSIFICAÇÃO, MAS DE REGRESSÃO
loss_fn = nn.MSELoss()

learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

epochs = 10000

# MINI-BATCH
batch_size = 50
batch_no = len(x_train) // batch_size



print("x_train, y_train")
print(x_train, y_train)

for epoch in range(epochs):
    for i in range(batch_no):

        start = i * batch_size
        end = start + batch_size

        x_var = x_train[start:end]
        y_var = y_train[start:end]

        optimizer.zero_grad()

        output_train = model(x_var)

        loss = loss_fn(output_train, y_var)

        if (epoch % (epochs / 10) == 0 and i == 0) :
            print(epoch, loss.item())

        loss.backward()

        optimizer.step()

y_pred = torch.argmax(model(x_test), dim = 1)

print(y_pred, y_test)

print((y_pred == y_test).sum()/len(y_test))

Tem nulos? False
x_train, y_train
tensor([[0.0025, 0.0421, 0.0316,  ..., 0.0418, 0.0421, 0.0260],
        [0.0061, 0.0211, 0.0421,  ..., 0.0573, 0.0571, 0.0343],
        [0.0042, 0.0421, 0.0526,  ..., 0.0501, 0.0499, 0.0311],
        ...,
        [0.0043, 0.0211, 0.0421,  ..., 0.0496, 0.0499, 0.0317],
        [0.0115, 0.0421, 0.0316,  ..., 0.0697, 0.0705, 0.0426],
        [0.0032, 0.0421, 0.0211,  ..., 0.0447, 0.0453, 0.0282]]) tensor([ 559., 2201., 1238.,  ..., 1169., 8364.,  526.])
0 50842952.0


c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([50])) that is different to the input size (torch.Size([50, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


KeyboardInterrupt: 

In [82]:
import seaborn as sns
import torch
import torch.nn as nn

# PEGANDO MENOS REGISTROS PORQUE SE NÃO MEU PC NÃO AGUENTA
diamonds = sns.load_dataset("diamonds")#.head(10000)


# NÃO HÁ NULOS, POR ISSO NÃO PRECISA TRATAR
print('Tem algum nulo? ' + str(diamonds.isna().any().any()))

# MAPS
map_cut = {'Fair':0,'Good':1,'Very Good':2,'Premium':3,'Ideal':4}
map_color = {'J':0,'I':1,'H':2,'G':3,'F':4,'E':5,'D':6}
map_clarity = {'I1':0,'SI2':1,'SI1':2,'VS2':3,'VS1':4,'VVS2':5,'VVS1':6,'IF':7}

train=diamonds.sample(frac=0.8,random_state=42)
test=diamonds.drop(train.index)

# TRANSFORMANDO EM NÚMERO
train["cut"] = train.cut.map(map_cut)
test["cut"]  = test.cut.map(map_cut)

train["color"] = train.color.map(map_color)
test["color"]  = test.color.map(map_color)

train["clarity"] = train.clarity.map(map_clarity)
test["clarity"]  = test.clarity.map(map_clarity)

# SEPARANDO AS COLUNAS
x_train = train.loc[ : ,  ["carat","cut","color","clarity","depth","table","x","y","z"]].values
x_test  = test.loc[ : , ["carat","cut","color","clarity","depth","table","x","y","z"]].values

# COLUNA QUE EU QUERO PREVER -> price -> dado um diamante, qual o preço dele?
# TARGET TREINO E TESTE
y_train = train.price.values
y_test  = test.price.values

# X
x_train = torch.tensor(x_train, dtype = torch.float)
x_test  = torch.tensor(x_test, dtype = torch.float)
# x_train = x_train.unsqueeze(1)
# x_test = x_test.unsqueeze(1)
# Y
y_train = torch.tensor(y_train, dtype = torch.float)
y_test  = torch.tensor(y_test, dtype = torch.float)
# y_train = y_train.unsqueeze(1)
# y_test = y_test.unsqueeze(1)

# NORMALIZANDO
x_train = (x_train - x_train.min())/(x_train.max() - x_train.min())
x_test  = (x_test  - x_train.min())/(x_train.max() - x_train.min())

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(NeuralNetwork, self).__init__()
        self.input_layer = nn.Linear(input_dim, 128)
        self.hidden_layer1 = nn.Linear(128, 64)
        self.hidden_layer2 = nn.Linear(64, 32)  
        self.output_layer = nn.Linear(32, output_dim)
        
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.relu(self.input_layer(x))
        out = self.relu(self.hidden_layer1(out))
        out = self.relu(self.hidden_layer2(out))
        return self.output_layer(out)

input_dim = x_train.shape[1]
output_dim = 1 ## PREÇO FINAL
model = NeuralNetwork(input_dim,output_dim)

# PROBLEMA NÃO É DE CLASSIFICAÇÃO, MAS DE REGRESSÃO
loss_fn = nn.MSELoss()

learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

epochs = 10000

# MINI-BATCH
batch_size = 10000000000
batch_no = len(x_train) // batch_size

for epoch in range(epochs):
    for i in range(batch_no):
        start = i * batch_size
        end = start + batch_size

        x_var = x_train[start:end]
        y_var = y_train[start:end]

        optimizer.zero_grad()

        output_train = model(x_var).squeeze(-1) # SQUEEZE PRA FICAR NA MESMA DIMENSÃO

        loss = loss_fn(output_train, y_var)

        if (epoch % (epochs / 10) == 0 and i == 0) :
            print(epoch, loss.item())

        loss.backward()
        optimizer.step()

y_pred = model(x_test)

medio = torch.mean(torch.abs(y_pred.squeeze() - y_test))
print('Erro Médio Absoluto:', medio.item())

porcentagem = 100 - (medio.item() / torch.mean(torch.abs(y_test)).item() * 100)
print('Porcentagem de Acerto:', porcentagem)


Tem algum nulo? False
Erro Médio Absoluto: 3919.083740234375
Porcentagem de Acerto: -0.006404370322371733


In [3]:
import seaborn as sns
import torch
import torch.nn as nn

# PEGANDO MENOS REGISTROS PORQUE SE NÃO MEU PC NÃO AGUENTA
diamonds = sns.load_dataset("diamonds")#.head(10000)

print('Tem algum nulo? ' + str(diamonds.isna().any().any()))

map_cut = {'Fair':0,'Good':1,'Very Good':2,'Premium':3,'Ideal':4}
map_color = {'J':0,'I':1,'H':2,'G':3,'F':4,'E':5,'D':6}
map_clarity = {'I1':0,'SI2':1,'SI1':2,'VS2':3,'VS1':4,'VVS2':5,'VVS1':6,'IF':7}

train=diamonds.sample(frac=0.8,random_state=42)
test=diamonds.drop(train.index)

train["cut"] = train.cut.map(map_cut)
test["cut"]  = test.cut.map(map_cut)

train["color"] = train.color.map(map_color)
test["color"]  = test.color.map(map_color)

train["clarity"] = train.clarity.map(map_clarity)
test["clarity"]  = test.clarity.map(map_clarity)

x_train = train.loc[ : ,  ["carat","cut","color","clarity","depth","table","x","y","z"]].values
x_test  = test.loc[ : , ["carat","cut","color","clarity","depth","table","x","y","z"]].values

y_train = train.price.values
y_test  = test.price.values

x_train = torch.tensor(x_train, dtype = torch.float)
x_test  = torch.tensor(x_test, dtype = torch.float)

y_train = torch.tensor(y_train, dtype = torch.float)
y_test  = torch.tensor(y_test, dtype = torch.float)

x_train = (x_train - x_train.min())/(x_train.max() - x_train.min())
x_test  = (x_test  - x_train.min())/(x_train.max() - x_train.min())

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim):
        super(NeuralNetwork, self).__init__()
        self.input_layer = nn.Linear(input_dim, 128)
        self.hidden_layer1 = nn.Linear(128, 64)
        self.output_layer = nn.Linear(64, 1)
        
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.relu(self.input_layer(x))
        out = self.relu(self.hidden_layer1(out))
        return self.output_layer(out)

input_dim = x_train.shape[1]
model = NeuralNetwork(input_dim)

loss_fn = nn.MSELoss()

learning_rate = 0.001 
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

epochs = 10000

batch_size = 1000000000
batch_no = len(x_train) // batch_size

for epoch in range(epochs):
    for i in range(batch_no):
        start = i * batch_size
        end = start + batch_size

        x_var = x_train[start:end]
        y_var = y_train[start:end]

        optimizer.zero_grad()

        output_train = model(x_var).squeeze(-1)

        loss = loss_fn(output_train, y_var)

        if (epoch % (epochs / 10) == 0 and i == 0) :
            print(epoch, loss.item())

        loss.backward()
        optimizer.step()

y_pred = model(x_test).squeeze(-1)

with open('y_pred.txt', 'w') as f:
    f.write(str(y_pred[:100]))

with open('y_test.txt', 'w') as f:
    f.write(str(y_test[:100]))

medio = torch.mean(torch.abs(y_pred.squeeze() - y_test))
print('Erro Médio Absoluto:', medio.item())

porcentagem = 100 - (medio.item() / torch.mean(torch.abs(y_test)).item() * 100)
print('Porcentagem de Acerto:', porcentagem)


Tem algum nulo? False
Erro Médio Absoluto: 3919.474609375
Porcentagem de Acerto: -0.016378491806918305


In [96]:
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Carregando o conjunto de dados
diamonds = sns.load_dataset("diamonds")

print('Tem algum nulo? ' + str(diamonds.isna().any().any()))

map_cut = {'Fair':0,'Good':1,'Very Good':2,'Premium':3,'Ideal':4}
map_color = {'J':0,'I':1,'H':2,'G':3,'F':4,'E':5,'D':6}
map_clarity = {'I1':0,'SI2':1,'SI1':2,'VS2':3,'VS1':4,'VVS2':5,'VVS1':6,'IF':7}

# Transformando dados
diamonds['cut'] = diamonds['cut'].map(map_cut)
diamonds['color'] = diamonds['color'].map(map_color)
diamonds['clarity'] = diamonds['clarity'].map(map_clarity)

# Separando características e alvo
X = diamonds[["carat", "cut", "color", "clarity", "depth", "table", "x", "y", "z"]].values
y = diamonds['price'].values

# Dividindo o conjunto em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizando os dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convertendo para tensores
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Definindo a arquitetura da rede neural
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim):
        super(NeuralNetwork, self).__init__()
        self.input_layer = nn.Linear(input_dim, 256)
        self.hidden_layer1 = nn.Linear(256, 128)
        self.dropout = nn.Dropout(0.2)  # Dropout de 20%
        self.output_layer = nn.Linear(128, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.relu(self.input_layer(x))
        out = self.dropout(out)
        out = self.relu(self.hidden_layer1(out))
        return self.output_layer(out)

# Inicializando o modelo
model = NeuralNetwork(input_dim=X_train.shape[1])

# Definindo a função de perda e o otimizador
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Configurando o agendador de taxa de aprendizado
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)

# Treinamento
epochs = 2000
best_loss = float('inf')

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = loss_fn(output, y_train)
    loss.backward()
    optimizer.step()
    scheduler.step(loss)  # Atualizando a taxa de aprendizado

    if epoch % 100 == 0:
        print(f'Epoch [{epoch}/{epochs}], Loss: {loss.item():.4f}')

    # Early stopping
    if loss.item() < best_loss:
        best_loss = loss.item()
        torch.save(model.state_dict(), 'best_model.pth')  # Salvando o melhor modelo

# Testando o modelo
model.eval()
y_pred = model(X_test)

# Erro médio absoluto
mae = torch.mean(torch.abs(y_pred.squeeze() - y_test.squeeze()))
print('Erro Médio Absoluto:', mae.item())

# Porcentagem de acerto
porcentagem = 100 - (mae.item() / torch.mean(torch.abs(y_test)).item() * 100)
print('Porcentagem de Acerto:', porcentagem)


Tem algum nulo? False
Epoch [0/2000], Loss: 31439196.0000
Epoch [100/2000], Loss: 28451764.0000
Epoch [200/2000], Loss: 10601800.0000
Epoch [300/2000], Loss: 3056218.5000
Epoch [400/2000], Loss: 1945246.0000


KeyboardInterrupt: 